In [ ]:
'''
First, we import all required packages.
'''
import random
from random import shuffle
random.seed(11)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(11)

In [ ]:
'''
Next, we read in the unlabeled data from a CSV file. Each line contains one example.
'''
def load_data(fname = 'BYOSC_data.csv'):
    data = []
    for line in open(fname, encoding="utf8", errors="ignore"):
        y, _, _, _, _, x = line[1:-2].strip().split('","')
        data.append((x, y))
    
    shuffle(data)
    
    train_set = data[:300]
    dev_set = data[300:]

    return train_set, dev_set

In [ ]:
'''
Methods needed to convert the data into input that can be fed into the neural network.
'''
def make_feature_vectors(data, w2i):
    new_data = []

    for (x, y) in data:
        sentence = []
        for word in x.split(' '):
            if word in w2i:
                sentence.append(w2i[word])

        new_data.append((torch.tensor(sentence), torch.tensor([int(int(y)/2)])))
        
    return new_data

def get_vocab(data):
    w2i = {'<UNK>': 0}
    
    for (x, y) in data:
        for word in x.split(' '):
            if word not in w2i:
                w2i[word] = len(w2i)

    return w2i

In [ ]:
'''
Now, we define our neural network model.
'''
class MyNNModel(nn.Module):

    def __init__(self, vocab_size, emb_dim, hidden_dim, output_dim):
        super(MyNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(emb_dim, hidden_dim)
        self.linear = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, input):
        x_1 = self.embedding(input)
        x_2 = self.lstm(x_1.unsqueeze(1))[1][0].squeeze(0).squeeze(0)
        return self.linear(x_2)

In [ ]:
'''
An evaluation method so we can see how well the model is doing.
'''
def eval(model, data):
    model.eval()
    
    total = right = 0
    for (x, y) in data:
        probs = F.softmax(model(x), dim=0)
        y_hat = torch.argmax(probs)
        if y_hat == y:
            right += 1
        total += 1
    print("Accuracy: " + str((right * 1.0)/total))

In [ ]:
'''
Finally, let's put everything together and train the model!
'''
train, dev = load_data()
vocab = get_vocab(train)
train = make_feature_vectors(train, vocab)
dev = make_feature_vectors(dev, vocab)
    
# TODO[3]: substitute -1 by more reasonable values!
model = MyNNModel(len(vocab), 64, 32, 3)
    
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=.01)
    
eval(model, train)
eval(model, dev)
print()
        
for i in range(3):
    model.train()
    for (x, y) in train:
        model.zero_grad()
        raw_scores = model(x)
        loss = loss_function(raw_scores.unsqueeze(0), y)
        loss.backward()
        optimizer.step()
        
    eval(model, train)
    eval(model, dev)
    print()